- http://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html
- http://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html
- http://alstatr.blogspot.com.br/2015/04/python-and-r-basic-sampling-problem.html
- http://stackoverflow.com/questions/591200/can-i-use-a-decorator-to-mutate-the-local-scope-of-a-function-in-python
- http://stackoverflow.com/questions/2001138/how-to-create-dynamical-scoped-variables-in-python
- http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-pipe

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
def _d1d2(S, K, T, r, v, q):
	d1 = ( np.log(S/K) + (r - q + 0.5*v**2)*T ) / (v*np.sqrt(T))
	return d1, d1 - v*np.sqrt(T)


def bsmprice(S, K, T, r, v, q):
	d1, d2 = _d1d2(S, K, T, r, v, q)
	N1 = norm.cdf(d1)
	N2 = norm.cdf(d2)
	return (S*np.exp(-q*T)*N1 - K*np.exp(-r*T)*N2, -S*np.exp(-q*T)*(1-N1) + K*np.exp(-r*T)*(1-N2))

Abordagem 1

In [3]:
options = pd.DataFrame(dict(
        Strike=np.linspace(50, 150, 10),
        Rate=0.1365,
        CouponRate=0,
        Volatility=0.4,
        Time=0.5
    ))

In [4]:
options

,CouponRate,Rate,Strike,Time,Volatility
0,0,0.1365,50.000000,0.5,0.4
1,0,0.1365,61.111111,0.5,0.4
2,0,0.1365,72.222222,0.5,0.4
3,0,0.1365,83.333333,0.5,0.4
4,0,0.1365,94.444444,0.5,0.4
5,0,0.1365,105.555556,0.5,0.4
6,0,0.1365,116.666667,0.5,0.4
7,0,0.1365,127.777778,0.5,0.4
8,0,0.1365,138.888889,0.5,0.4
9,0,0.1365,150.000000,0.5,0.4


In [12]:
options['Spot'] = options.loc[5, 'Strike']*np.exp(-options['Rate']*options['Time'])

In [13]:
options

,CouponRate,Rate,Strike,Time,Volatility,Spot
0,0,0.1365,50.000000,0.5,0.4,98.591732
1,0,0.1365,61.111111,0.5,0.4,98.591732
2,0,0.1365,72.222222,0.5,0.4,98.591732
3,0,0.1365,83.333333,0.5,0.4,98.591732
4,0,0.1365,94.444444,0.5,0.4,98.591732
5,0,0.1365,105.555556,0.5,0.4,98.591732
6,0,0.1365,116.666667,0.5,0.4,98.591732
7,0,0.1365,127.777778,0.5,0.4,98.591732
8,0,0.1365,138.888889,0.5,0.4,98.591732
9,0,0.1365,150.000000,0.5,0.4,98.591732


In [18]:
_premia = bsmprice(
    options['Spot'],
    options['Strike'],
    options['Time'],
    options['Rate'],
    options['Volatility'],
    options['CouponRate']
)

options['Call'] = _premia[0]
options['Put'] = _premia[1]

In [19]:
options

,CouponRate,Rate,Strike,Time,Volatility,Spot,Call,Put
0,0,0.1365,50.000000,0.5,0.4,98.591732,51.914755,0.024369
1,0,0.1365,61.111111,0.5,0.4,98.591732,41.726205,0.213897
2,0,0.1365,72.222222,0.5,0.4,98.591732,32.087986,0.953755
3,0,0.1365,83.333333,0.5,0.4,98.591732,23.534728,2.778574
4,0,0.1365,94.444444,0.5,0.4,98.591732,16.488908,6.110830
5,0,0.1365,105.555556,0.5,0.4,98.591732,11.087914,11.087914
6,0,0.1365,116.666667,0.5,0.4,98.591732,7.200050,17.578127
7,0,0.1365,127.777778,0.5,0.4,98.591732,4.543179,25.299333
8,0,0.1365,138.888889,0.5,0.4,98.591732,2.801664,33.935895
9,0,0.1365,150.000000,0.5,0.4,98.591732,1.696965,43.209274


Abordagem 2

In [20]:
options = pd.DataFrame(dict(
        Strike=np.linspace(50, 150, 10),
        Rate=0.1365,
        CouponRate=0,
        Volatility=0.4,
        Time=0.5
    ))

In [25]:
options['Spot'] = options.apply(lambda o: options.loc[5, 'Strike']*np.exp(-o.Rate*o.Time), axis='columns')

In [26]:
options

,CouponRate,Rate,Strike,Time,Volatility,Spot
0,0,0.1365,50.000000,0.5,0.4,98.591732
1,0,0.1365,61.111111,0.5,0.4,98.591732
2,0,0.1365,72.222222,0.5,0.4,98.591732
3,0,0.1365,83.333333,0.5,0.4,98.591732
4,0,0.1365,94.444444,0.5,0.4,98.591732
5,0,0.1365,105.555556,0.5,0.4,98.591732
6,0,0.1365,116.666667,0.5,0.4,98.591732
7,0,0.1365,127.777778,0.5,0.4,98.591732
8,0,0.1365,138.888889,0.5,0.4,98.591732
9,0,0.1365,150.000000,0.5,0.4,98.591732


In [31]:
options['Call'] = options.apply(lambda o: bsmprice(o.Spot, o.Strike, o.Time, o.Rate, o.Volatility, o.CouponRate)[0], axis='columns')
options['Put'] = options.apply(lambda o: bsmprice(o.Spot, o.Strike, o.Time, o.Rate, o.Volatility, o.CouponRate)[1], axis='columns')

In [32]:
options

,CouponRate,Rate,Strike,Time,Volatility,Spot,Call,Put
0,0,0.1365,50.000000,0.5,0.4,98.591732,51.914755,0.024369
1,0,0.1365,61.111111,0.5,0.4,98.591732,41.726205,0.213897
2,0,0.1365,72.222222,0.5,0.4,98.591732,32.087986,0.953755
3,0,0.1365,83.333333,0.5,0.4,98.591732,23.534728,2.778574
4,0,0.1365,94.444444,0.5,0.4,98.591732,16.488908,6.110830
5,0,0.1365,105.555556,0.5,0.4,98.591732,11.087914,11.087914
6,0,0.1365,116.666667,0.5,0.4,98.591732,7.200050,17.578127
7,0,0.1365,127.777778,0.5,0.4,98.591732,4.543179,25.299333
8,0,0.1365,138.888889,0.5,0.4,98.591732,2.801664,33.935895
9,0,0.1365,150.000000,0.5,0.4,98.591732,1.696965,43.209274


In [37]:
def price(o):
    call, put = bsmprice(o.Spot, o.Strike, o.Time, o.Rate, o.Volatility, o.CouponRate)
    return dict(call=call, put=put)

options.apply(price, axis='columns')

0    <built-in method values of dict object at 0x11...
1    <built-in method values of dict object at 0x11...
2    <built-in method values of dict object at 0x11...
3    <built-in method values of dict object at 0x11...
4    <built-in method values of dict object at 0x11...
5    <built-in method values of dict object at 0x11...
6    <built-in method values of dict object at 0x11...
7    <built-in method values of dict object at 0x11...
8    <built-in method values of dict object at 0x11...
9    <built-in method values of dict object at 0x11...
dtype: object

In [36]:
import inspect

inspect.getargspec(bsmprice)

ArgSpec(args=['S', 'K', 'T', 'r', 'v', 'q'], varargs=None, keywords=None, defaults=None)

Fazer um *bind* do DataFrame a funções e métodos através de um decorator.

- datadecorators
- datadec



In [41]:
# np.vectorize

vbsmprice = np.vectorize(bsmprice)
options['Call'], options['Put'] = vbsmprice(options.Spot, options.Strike, options.Time, options.Rate, options.Volatility, options.CouponRate)
options

,CouponRate,Rate,Strike,Time,Volatility,Spot,Call,Put
0,0,0.1365,50.000000,0.5,0.4,98.591732,51.914755,0.024369
1,0,0.1365,61.111111,0.5,0.4,98.591732,41.726205,0.213897
2,0,0.1365,72.222222,0.5,0.4,98.591732,32.087986,0.953755
3,0,0.1365,83.333333,0.5,0.4,98.591732,23.534728,2.778574
4,0,0.1365,94.444444,0.5,0.4,98.591732,16.488908,6.110830
5,0,0.1365,105.555556,0.5,0.4,98.591732,11.087914,11.087914
6,0,0.1365,116.666667,0.5,0.4,98.591732,7.200050,17.578127
7,0,0.1365,127.777778,0.5,0.4,98.591732,4.543179,25.299333
8,0,0.1365,138.888889,0.5,0.4,98.591732,2.801664,33.935895
9,0,0.1365,150.000000,0.5,0.4,98.591732,1.696965,43.209274


# using assign

In [45]:
options = pd.DataFrame(dict(
        Strike=np.linspace(50, 150, 10),
        Rate=0.1365,
        CouponRate=0,
        Volatility=0.4,
        Time=0.5
    ))

In [89]:
# options['Spot'] = options.apply(lambda o: options.loc[5, 'Strike']*np.exp(-o.Rate*o.Time), axis='columns')
# func = lambda: Strike[5]*np.exp(-Rate*Time)
options.assign(Spot=lambda o: o.Strike[5]*np.exp(-o.Rate*o.Time))

,CouponRate,Rate,Strike,Time,Volatility,Spot
0,0,0.1365,50.000000,0.5,0.4,98.591732
1,0,0.1365,61.111111,0.5,0.4,98.591732
2,0,0.1365,72.222222,0.5,0.4,98.591732
3,0,0.1365,83.333333,0.5,0.4,98.591732
4,0,0.1365,94.444444,0.5,0.4,98.591732
5,0,0.1365,105.555556,0.5,0.4,98.591732
6,0,0.1365,116.666667,0.5,0.4,98.591732
7,0,0.1365,127.777778,0.5,0.4,98.591732
8,0,0.1365,138.888889,0.5,0.4,98.591732
9,0,0.1365,150.000000,0.5,0.4,98.591732


In [52]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [58]:
func.__code__

<code object <lambda> at 0x113111780, file "<ipython-input-48-713f0fb2f5e1>", line 2>

In [63]:
options.to_dict('series')

{'CouponRate': 0    0
 1    0
 2    0
 3    0
 4    0
 5    0
 6    0
 7    0
 8    0
 9    0
 Name: CouponRate, dtype: int64, 'Rate': 0    0.1365
 1    0.1365
 2    0.1365
 3    0.1365
 4    0.1365
 5    0.1365
 6    0.1365
 7    0.1365
 8    0.1365
 9    0.1365
 Name: Rate, dtype: float64, 'Strike': 0     50.000000
 1     61.111111
 2     72.222222
 3     83.333333
 4     94.444444
 5    105.555556
 6    116.666667
 7    127.777778
 8    138.888889
 9    150.000000
 Name: Strike, dtype: float64, 'Time': 0    0.5
 1    0.5
 2    0.5
 3    0.5
 4    0.5
 5    0.5
 6    0.5
 7    0.5
 8    0.5
 9    0.5
 Name: Time, dtype: float64, 'Volatility': 0    0.4
 1    0.4
 2    0.4
 3    0.4
 4    0.4
 5    0.4
 6    0.4
 7    0.4
 8    0.4
 9    0.4
 Name: Volatility, dtype: float64}

In [78]:
def bind(func, **data):
    locals().update(data)
    return lambda: func()

bfunc = bind(lambda: a+b, a=1, b=2)
bfunc()

NameError: name 'a' is not defined

In [87]:
def add_name(name):
    def inner(func):
        func.__globals__['name'] = name
        return func
    return inner
@add_name('Wilson')
def say_hello():
    print("Hello %s!" % name)

say_hello()

Hello Wilson!


In [88]:
name

'Wilson'